In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter

Supervised NLP requires a pre-labelled dataset for training and testing, and is generally interested in categorizing text in various ways. In this case, we are going to try to predict whether a sentence comes from _Alice in Wonderland_ by Lewis Carroll or _Hamlet_ by Shakespeare. We can use any of the supervised models we've covered previously, as long as they allow categorical outcomes. In this case, we'll try Random Forests, SVM, and KNN.

Our feature-generation approach will be something called _BoW_, or _Bag of Words_. BoW is quite simple: For each sentence, we count how many times each word appears. We will then use those counts as features.  

In [2]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text
    
# Load and clean the data.
shakespeare = gutenberg.raw('shakespeare-hamlet.txt')
alice = gutenberg.raw('carroll-alice.txt')

# The Chapter indicator is idiosyncratic
shakespeare = re.sub(r'Chapter \d+', '', shakespeare)
alice = re.sub(r'CHAPTER .*', '', alice)
    
alice = text_cleaner(alice)
shakespeare = text_cleaner(shakespeare)

In [3]:
# Parse the cleaned novels. This can take a bit.
nlp = spacy.load('en')
alice_doc = nlp(alice)
shakespeare_doc = nlp(shakespeare)

In [4]:
# Group into sentences.
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
shakespeare_sents = [[sent, "Shakespeare"] for sent in shakespeare_doc.sents]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(alice_sents + shakespeare_sents)
sentences.head()

,0,1
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(I, shall, be, late, !, ')",Carroll


Time to bag some words!  Since spaCy has already tokenized and labelled our data, we can move directly to recording how often various words occur.  We will exclude stopwords and punctuation.  In addition, in an attempt to keep our feature space from exploding, we will work with lemmas (root words) rather than the raw text terms, and we'll only use the 2000 most common words for each text.

In [5]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
    return df

# Set up the bags.
alicewords = bag_of_words(alice_doc)
shakespearewords = bag_of_words(shakespeare_doc)

# Combine bags to create a set of unique words.
common_words = set(alicewords + shakespearewords)

In [6]:
# Create our data frame with features. This can take a while to run.
word_counts = bow_features(sentences, common_words)
word_counts.head()

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 3500
Processing row 4000
Processing row 4500


,mouth,dreadfully,angels,disobey,somebody,lookt,opinion,doubt,justice,sodaine,...,arrange,stop,oft,nunnery,toy,sand,globe,vnction,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(I, shall, be, late, !, ')",Carroll


## Trying out BoW

Now let's give the bag of words features a whirl by trying a random forest.

In [8]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split

rfc = ensemble.RandomForestClassifier()
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)
train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

Training set score: 0.9851274787535411

Test set score: 0.8747346072186837


Holy overfitting, Batman! Overfitting is a known problem when using bag of words, since it basically involves throwing a massive number of features at a model – some of those features (in this case, word frequencies) will capture noise in the training set. Since overfitting is also a known problem with Random Forests, the divergence between training score and test score is expected.


## BoW with Logistic Regression

Let's try a technique with some protection against overfitting due to extraneous features – logistic regression with lasso.

In [9]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

(2824, 3354) (2824,)
Training set score: 0.9631728045325779

Test set score: 0.9129511677282378


Logistic regression performs a bit better than the random forest.  

# BoW with Gradient Boosting

And finally, let's see what gradient boosting can do:

In [10]:
clf = ensemble.GradientBoostingClassifier()
train = clf.fit(X_train, y_train)

print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))

Training set score: 0.8764164305949008

Test set score: 0.8328025477707006


Looks like logistic regression is the winner, but there's room for improvement.

# Same model, new inputs

What if we feed the model a different play by Shakespeare, like _Macbeth_?  Will it be able to distinguish Shakespeare from Carroll with the same level of accuracy if we insert a different sample of Shakespeare's writing?

First, we need to process _Macbeth_ the same way we processed the other data, and combine it with the Alice data:

In [11]:
# Clean the Emma data.
macbeth = gutenberg.raw('shakespeare-macbeth.txt')
macbeth = re.sub(r'VOLUME \w+', '', macbeth)
macbeth = re.sub(r'CHAPTER \w+', '', macbeth)
macbeth = text_cleaner(macbeth)
print(macbeth[:100])

Actus Primus. Scoena Prima. Thunder and Lightning. Enter three Witches. 1. When shall we three meet 


In [12]:
# Parse our cleaned data.
macbeth_doc = nlp(macbeth)

In [16]:
# Group into sentences.
shakespeare_sents = [[sent, "Shakespeare"] for sent in shakespeare_doc.sents]
macbeth_sents = [[sent, "Shakespeare"] for sent in macbeth_doc.sents]

# Macbeth is quite long, let's cut it down to the same length as Alice.
#macbeth_sents = macbeth_sents[0:len(alice_sents)]

In [17]:
# Build a new Bag of Words data frame for Macbeth word counts.
# We'll use the same common words from Alice and Macbeth.
macbeth_sentences = pd.DataFrame(macbeth_sents)
macbeth_bow = bow_features(macbeth_sentences, common_words)

print('done')

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
done


In [18]:
# Now we can model it!
# Let's use logistic regression again.

# Combine the Macbeth sentence data with the Alice data from the test set.
X_Macbeth_test = np.concatenate((
    X_train[y_train[y_train=='Carroll'].index],
    macbeth_bow.drop(['text_sentence','text_source'], 1)
), axis=0)
y_Macbeth_test = pd.concat([y_train[y_train=='Carroll'],
                         pd.Series(['Shakespeare'] * macbeth_bow.shape[0])])

# Model.
print('\nTest set score:', lr.score(X_Macbeth_test, y_Macbeth_test))
lr_Macbeth_predicted = lr.predict(X_Macbeth_test)
pd.crosstab(y_Macbeth_test, lr_Macbeth_predicted)


Test set score: 0.7504320774282751


col_0,Carroll,Shakespeare
row_0,,
Carroll,320,666
Shakespeare,56,1851


Well look at that!  NLP approaches are generally effective on the same type of material as they were trained on. It looks like this model is actually able to differentiate multiple works by Austen from Alice in Wonderland.  Now the question is whether the model is very good at identifying Shakespeare, or very good at identifying Alice in Wonderland, or both...

# Challenge 0:

Recall that the logistic regression model's best performance on the test set was 91%.  See what you can do to improve performance.  Suggested avenues of investigation include: Other modeling techniques (SVM?), making more features that take advantage of the spaCy information (include grammar, phrases, POS, etc), making sentence-level features (number of words, amount of punctuation), or including contextual information (length of previous and next sentences, words repeated from one sentence to the next, etc), and anything else your heart desires.  Make sure to design your models on the test set, or use cross_validation with multiple folds, and see if you can get accuracy above 97%.  

### Support Vector Classifier

In [19]:
from sklearn.svm import SVC

svc = SVC()
svc.fit(X_train, y_train)

print('Training set score:', svc.score(X_train, y_train))
print('\nTest set score:', svc.score(X_test, y_test))

Training set score: 0.6508498583569405

Test set score: 0.6374734607218684


## DataFrame with Nouns

In [20]:
# List of each noun in Shakespeare's sentences
shakespeare_noun_list = []
for chunk in shakespeare_doc.noun_chunks:
    shakespeare_noun_list.append(chunk.root.text)
shakespeare_nouns = list(set(shakespeare_noun_list))

# List of each noun in Alice's sentences
alice_noun_list = []
for chunk in alice_doc.noun_chunks:
    alice_noun_list.append(chunk.root.text)
alice_nouns = list(set(alice_noun_list))

In [21]:
# Creates a data frame with features for each noun in our noun list set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, noun_list):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=noun_list)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, noun_list] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in noun_list
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
    return df


# Combine each list of nouns to create a set of unique words.
noun_list = set(alice_nouns + shakespeare_nouns)

In [22]:
# Create our data frame with features. This can take a while to run.
noun_counts = bow_features(sentences, noun_list)
noun_counts.head()

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 3500
Processing row 4000
Processing row 4500


,mouth,games,somebody,jaws,whereat,disclaiming,Paconcies,eielids,opinion,doubt,...,Hazard,stop,Spurnes,toy,sand,globe,Stages,turtles,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(I, shall, be, late, !, ')",Carroll


## Use each model with this new Noun DataFrame

#### Random Forest Classifier

In [23]:
rfc = ensemble.RandomForestClassifier()
Y = noun_counts['text_source']
X = np.array(noun_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)
train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

Training set score: 0.9213881019830028

Test set score: 0.8253715498938429


#### Logistic Regression

In [24]:
lr = LogisticRegression()
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

(2824, 3568) (2824,)
Training set score: 0.8870396600566572

Test set score: 0.8306794055201698


#### Gradient Boosting Classifier

In [25]:
clf = ensemble.GradientBoostingClassifier()
train = clf.fit(X_train, y_train)

print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))

Training set score: 0.8016997167138811

Test set score: 0.7669851380042463


#### Support Vector Classifier

In [26]:
svc = SVC()
svc.fit(X_train, y_train)

print('Training set score:', svc.score(X_train, y_train))
print('\nTest set score:', svc.score(X_test, y_test))

Training set score: 0.6508498583569405

Test set score: 0.6374734607218684


# Challenge 1:
Find out whether your new model is good at identifying Alice in Wonderland vs any other work, Persuasion vs any other work, or Austen vs any other work.  This will involve pulling a new book from the Project Gutenberg corpus (print(gutenberg.fileids()) for a list) and processing it.

Record your work for each challenge in a notebook and submit it below.

In [27]:
print(gutenberg.fileids())

['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']


In [28]:
# Clean the Emma data.
bryant_stories = gutenberg.raw('bryant-stories.txt')
bryant_stories = re.sub(r'VOLUME \w+', '', bryant_stories)
bryant_stories = re.sub(r'CHAPTER \w+', '', bryant_stories)
bryant_stories = text_cleaner(bryant_stories)
print(bryant_stories[:100])

TWO LITTLE RIDDLES IN RHYME There's a garden that I ken, Full of little gentlemen; Little caps of bl


In [29]:
# Parse our cleaned data.
bryant_stories_doc = nlp(bryant_stories)

In [30]:
# Group into sentences.
bryant_sents = [[sent, "Bryant"] for sent in bryant_stories_doc.sents]
shakespeare_sents = [[sent, "Shakespeare"] for sent in shakespeare_doc.sents]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(bryant_sents + shakespeare_sents)
sentences.head()

,0,1
0,"(TWO, LITTLE, RIDDLES, IN, RHYME)",Bryant
1,"(There, 's, a, garden, that, I, ken, ,, Full, ...",Bryant
2,"((, Flax, ., ))",Bryant
3,"(From, house, to, house, he, goes, ,, A, messe...",Bryant
4,"((, The, path, ., ))",Bryant


In [35]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
    return df

# Set up the bags.
bryantwords = bag_of_words(bryant_stories_doc)
shakespearewords = bag_of_words(shakespeare_doc)

# Combine bags to create a set of unique words.
common_words = set(bryantwords + shakespearewords)

In [36]:
# Create our data frame with features. This can take a while to run.
word_counts = bow_features(sentences, common_words)
word_counts.head()

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 3500
Processing row 4000
Processing row 4500
Processing row 5000
Processing row 5500


,mouth,angels,rashkill,pane,somebody,lookt,opinion,doubt,publisher,sodaine,...,stop,oft,nunnery,toy,sand,jewel,vnction,farrum,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(TWO, LITTLE, RIDDLES, IN, RHYME)",Bryant
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(There, 's, a, garden, that, I, ken, ,, Full, ...",Bryant
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"((, Flax, ., ))",Bryant
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(From, house, to, house, he, goes, ,, A, messe...",Bryant
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"((, The, path, ., ))",Bryant


### Random Forest Classifier

In [37]:
rfc = ensemble.RandomForestClassifier()
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)
train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

Training set score: 0.9859598853868194

Test set score: 0.8578178694158075


### Logistic Regression

In [38]:
lr = LogisticRegression()
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

(3490, 3432) (3490,)
Training set score: 0.9679083094555874

Test set score: 0.9183848797250859


### Gradient Boosting Classifier

In [39]:
clf = ensemble.GradientBoostingClassifier()
train = clf.fit(X_train, y_train)

print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))

Training set score: 0.856160458452722

Test set score: 0.8208762886597938


### Support Vector Classifier

In [40]:
svc = SVC()
svc.fit(X_train, y_train)

print('Training set score:', svc.score(X_train, y_train))
print('\nTest set score:', svc.score(X_test, y_test))

Training set score: 0.5232091690544413

Test set score: 0.5253436426116839


## Conclusion

During this challenge I saw repeatedly that over fitting is a consistent problem with Natural Language Processing. In my first drill for this challenge I attempted to use an SVM model but is had the worse performance out of all the models. I then decided to create a new dataframe with only the most common nouns for each book and see if that could change the accuracy for the models but it overall had a worse performance than the dataframe with the most common words. 

For the second drill in this challenge I used the Shakespeare and Bryant books to see how the models would perform with distinguishing these two books. The Logistic Regression again performed the best with a test score of 0.91 but the models were again over fitting. 

I am looking forward to getting more experience with spaCy's features because there are so many tools available that will help me analyze and model written texts. 